# Linear regression

The materials used in this tutorial are based on the applied exercises provided in the book <font color="orange">"An Introduction to Statistical Learning with Applications in R"</font> (ISLR). We are trying to demonstrate how to use R to carry out linear regression analysis on real-world datasets. Besides the exercises that we are going to cover in this tutorial, it is worth trying the other applied exercises by yourself.

## Question 1 Analyzing the Auto data set with Multiple Linear Regression

The first question is about applying multiple linear regression to analyzing the **Auto** dataset. You are going to 
* explore the correlation between variables, 
* use <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/lm.html">lm()</a> to build a multiple linear regression with <font color="orange">mpg</font> as the response variable. Building the regression model is just one simple step. You will learn to interpret the output of the <font color="blue">lm()</font> function using the <a href="https://stat.ethz.ch/R-manual/R-devel/library/base/html/summary.html">summary()</a> function.
* use the <font color="blue">plot()</font> function to produce diagnostic plots, e.g., residual plot and QQ plot.
* explore possible interaction between variables
* study how different variable transformations will affect the model.

Firstly, let's start with importing the libraries.

In [ ]:
library(ISLR)
library(ggplot2)
library(GGally)
library(gridExtra)
library(cowplot)

Next, the data should be loaded and inspected.

In [ ]:
# If you downloaded and stored the CSV locally in your computer
# Auto = read.csv("Auto.csv", header = T, na.strings = "?")
# If not, you can use the read.csv() function to read the file directly from the web.
#Auto = read.csv("http://www-bcf.usc.edu/~gareth/ISL/Auto.csv", header = T, na.strings = "?")
#Auto = na.omit(Auto) # if there are missing values.
head(Auto)

The dataset contains 392 observations on the following 9 variables. (The following detail is copied and pasted from <a href="https://cran.r-project.org/web/packages/ISLR/ISLR.pdf">here</a>)
* <font color="orange">mpg</font> miles per gallon
* <font color="orange">cylinders</font> Number of cylinders between 4 and 8 displacement Engine displacement (cu. inches)
* <font color="orange">horsepower</font> Engine horsepower
* <font color="orange">weight</font> Vehicle weight (lbs.)
* <font color="orange">acceleration</font> Time to accelerate from 0 to 60 mph (sec.)
* <font color="orange">year</font> Model year (modulo 100)
* <font color="orange">origin</font> Origin of car (1. American, 2. European, 3. Japanese)
* <font color="orange">name</font> Vehicle name

Assume that we are not interested in <font color="orange">name</font>. 
Here, <font color="orange">name</font> is actually a categorical variable. In the following task, we will not use it, so here we delete it from the dataframe. It is quite simple to delete the columns that you don't want. 

In [ ]:
Auto$name = NULL # drop name

In [ ]:
head(Auto)

You see that we always check the data after each manipulation, which is a very good practice in real data analysis. You should always be aware of what your data looks like.

We can have a look at the structure of <font color="orange">Auto</font> using the <a href="https://stat.ethz.ch/R-manual/R-devel/library/utils/html/str.html">str()</a> function. The function will give us information of the data that you should know. For example, the data type of each column, the total number of observations, the total number of variables, etc.

In [ ]:
str(Auto)

### 1.1 Produce a scatter plot matrix which includes all of the variables in the data set. 
In this task, you are going to generate pairwise scatter plots to explore the relationship between any two variables. The function that you will use is <a href="https://stat.ethz.ch/R-manual/R-devel/library/graphics/html/pairs.html">pairs()</a> to see its documentation, where you can find the specification of each argument. Click the hyper-link. You will find the API documentation of this function.

Try to explain the plots by finding some interesting relationships. For example, you will find 
 * a negative correlation between <font color='orange'>mpg</font> and  <font color='orange'>horsepower</font>
 * a positive correlation between  <font color='orange'>horsepower</font> and  <font color='orange'>weight</font>
 
You will find that pairwise scatter plots reveals a lot of information that we need while we are developing our model.

### 1.2 Compute the matrix of correlations between the variables using the function cor(). 
In the previous task, we explore the relationship between two variable with scatter plots. Now, the question is whether or not we can quantify the correlation between each pair of variables. Here we are going to use the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/cor.html">cor()</a>, which will generate a square matrix, where columns and rows are variables in the dataset. 

In the matrix, we will see that some values are positive and the other are negative, and all the values are in the range of -1 to +1. What do the values mean?

* A positive value indicates two variables are positively correlated. That is to say, for any two variables X and Y, an increase in X is associated with an increase in Y as well. The bigger the value, the stronger the positive correlation. 1 shows a perfect, linear positive correlation.
* Zero indicates there is no correlation between two variables.
* A negative value indicates two variables are negatively correlated. In other words, for any two variables X and Y, an increase in X is associated with a decrease in Y. -1 shows a perfect, linear negative correlation.

For more information, please read the following two webpages.
 1. What does a negative correlation coefficient mean?  http://www.investopedia.com/ask/answers/041015/what-does-negative-correlation-coefficient-mean.asp#ixzz4GVtQC8Eu 
 2. What does a negative correlation coefficient mean?  http://www.investopedia.com/ask/answers/041015/what-does-negative-correlation-coefficient-mean.asp#ixzz4GVtIDfM3 

We can also visualize the correlation matrix. The R library we are going to use is <a href="https://stat.ethz.ch/R-manual/R-devel/library/lattice/html/Lattice.html">lattice</a>. (Click the hyper-link and see more details of the lattice package, and look for <font color= "blue">levelplot()</font>). Here you are going to use the <font color= "blue">levelplot()</font> function by passing the output of the <font color= "blue">cor()</font> function as an argument.

In [ ]:
library(lattice)

The above plot shows that some variables are either positively or negatively correlated. 
We can also add the quantified correlation values to the plot, change the color scheme, add a title, etc.

### 1.3 Use the lm() function to perform a multiple linear regression with mpg as the response and all other variables as the predictors.

In this task, we are going to build a multiple linear regression model to predict <font color='orange'>mpg</font> (miles per gallon) by using all the other variables as predictors. By interpreting the results given by the <font color="blue">summary()</font>, you should be able to answer the following questions 
* Is there a relationship between the predictors and the response?
* Which predictors appear to have a statistically significant relationship to the response?
* What does the coefficient for the year variable suggest?

Let's build the regression model with the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/lm.html">lm()</a> function. The formula is "<font color="red">mpg~.</a>"

where <font color="red">"mpg~."</font> means the model regresses <font color="orange">mpg</font> on all the other variables. Now, we can print the result of our model with the <font color="blue">summary()</font> function

The output contains: residuals, coefficients, residual standard error, $R^2$, and F-statistic. They are the statistics that you need to assess the accuracy of your model. 

For example, the p-values shows that there is not a statistically significant relationship between <font color='orange'>mpg</font> and <font color='orange'>horsepower</font>, and between <font color='orange'>mpg</font> and <font color='orange'>acceleration</font>. In this case, we might choose to exclude the variables in the model.

In [ ]:
summary(lm(mpg~., data = subset(Auto, select=c( -horsepower, -acceleration ))))

There are many other ways of excluding specific set of variable from your linear model. For example,
```R
summary(lm(mpg~.-horsepower-acceleration, data = Auto))
```

Actually, you can compare the two summaries from the full model and the one excluding <font color="orange">horsepower</font> and <font color='orange'>acceleration</font>. The Adjusted R-squared for the full model is 0.8182, the one for the second model is 0.8166. There is not much difference between the two values. In other words, excluding the two variables makes the model simple but without losing the modeling accuracy.

Let's look at the F-statistic. F-statistic is a good indicator of whether there is a relationship between our predictor and the response variables. The further the F-statistic is from 1 the better it is.
The p-values shows that both models are statistically different from the constant model.  

### 1.4 Use the plot() function to produce diagnostic plots of the linear regression fit.
In this task, we will further diagnose your linear model with different plot. As discussed in the lecture, we need to check the following assumptions:
* <font color="red">Constant variance</font>
* <font color="red">linearity</font> 
* <font color="red">Normality</font>


In order to do so, we need the <font color = "blue">plot()</font> function. Without specifying which plot to show, the function will give us four diagnostic plots as follows. The argument should be the object returned by lm(). 

In [ ]:
par(mfcol=c(2,2)) # generate 2*2 plots

The diagnostic plots show residuals in four different ways.

1. The **residual vs fitted plot**: This plot is used to check the linear assumption. It shows if residuals have non-linear patterns. If you find equally spread residuals around a horizontal line without distinct patterns, that is a good indication you have linear relationships. However, if the relationship between predictors and an response variable is non-linear, an obvious pattern could show up in this plot if the model cannot capture the non-linearity. The first plot above shows that there could be a non-linear relationship between <font color='orange'>mpg </font>and all the predictors, as the residuals are not scattered evenly.
* The normal **Q-Q plot**: The Q-Q plot (i.e., quantile-quantile plot) is a graphical tool to help us assess if a set of data plausibly came from some theoretical distribution such as a Normal. For example, if we run a statistical analysis that assumes our dependent variable is Normally distributed, we can use a Normal Q-Q plot to check that assumption. In the case of linear regression analysis, we assume that residual is normally distributed with constant variance and mean equal to zero. The normal Q-Q plot shows if residuals are normally distributed. Do residuals follow a straight line well or do they deviate severely? It is good if residuals are lined well on the straight dashed line.  What do you think about the normal Q-Q plot above? Of course it cannot be a perfect straight line and this will be your call. it seems that the residuals are distributed normally. 
* The **scale-location plot**: It is used to check the assumption of equal variance by showing if residuals are spread equally along the ranges of predictors. It is good if we can see a horizontal line with equally (randomly) spread points. The scale-location plot shows that the residuals appear randomly spread.

* The **residual-leverage plot**: it helps us identify influential data samples. Not all outliers are influential in linear regression analysis. Here we care about the samples that are influential to determine the regression line. These samples can very influential even if they look to be within a reasonable range of the values. They can alter the results if we exclude them from analysis. In the residual-leverage plot, we look for outlying values at the upper right corner or at the lower right corner. Samples located in those places can be influential against a regression line. How do we identify those samples? We usually use Cook’s distance, indicated by a red dash line. When samples are outside of the Cook’s distance (i.e, they have high Cook’s distance scores, Cook’s distance measures how much the entire regression function changes when the i-th case is deleted.), the samples are influential to the regression results. The regression results will be altered if we exclude those samples. The fourth plot shows some outliers, such as 14. However, we can barely see Cook’s distance lines because all cases are well inside of the Cook’s distance lines. Therefore no influential cases are observed.

You can also choose to just show one of the four plots. For example, if you would like to print the residual plot for the residual analysis, add one more argument to the plot() function, which is "<font color="red">which = 1</font>".

### 1.5 Use the ":" symbol to fit linear regression models with interaction effects.
Do any interactions appear to be statistically significant? Here we would like to explore some possible interaction between variables. For example, let's try to check whether or not the interaction between <font color="orange">horsepower</font> and <font color="orange">weight</font> can affect the model accuracy. Instead of listing all the predictors, we can update the linear model fitted in Question 1.3. The function that you are going to use is <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/update.html">update()</a>.

In [ ]:
my_fit.update <- 
summary(my_fit.update)

You can compare the summary above with that generated by the model without the interaction terms. You will find the model with the interaction term explains more variation of the data than the one without interaction, see the Adjusted R-squared score.

We can also use the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/anova.lm.html">anova()</a> function to compare models. Note that the <font color="blue">anova()</font> function has one strong requirement when comparing two models: <font color="red">one model must be contained within the other</font>. In other words, all the terms of the smaller model must appear in the larger model. Otherwise, the comparison is impossible.

In [ ]:
anova(my_fit, my_fit.update)

The p-value shows that the two models are statistically different as The p-value is very low. It means that adding the interaction term does make a difference.

Try the following script, which transforms <font color="orange"> horsepower</font> with a log() function.

In [ ]:
summary(update(my_fit, . ~ . - horsepower + log(horsepower)))

Try to explain the summary by yourself.

## Question 2.  Analyzing sales of child car seats at 400 different stores.  

The following questions should be answered using the Carseats data set packaged in the ISLR library. In this task, we will learn how to handle qualitative variables (or categorical variables) in linear regression. We already imported the <font color="red">ISLR</font> library. Next, the <a href="http://127.0.0.1:12351/library/ISLR/html/Carseats.html">Carseats</a> data should be loaded and inspected. Similar to what we have done in question 1, we can use, for example, the <font color="blue">head()</font> function and the <font color="blue">str()</font>.

As you can see, the data frame contains 400 observations, each of which has 11 variables. (The following detail is copied and pasted from <a href="https://cran.r-project.org/web/packages/ISLR/ISLR.pdf">here</a>)
* <font color="orange">Sales</font> Unit sales (in thousands) at each location
* <font color="orange">CompPrice</font> Price charged by competitor at each location
* <font color="orange">Income</font> Community income level (in thousands of dollars)
* <font color="orange">Advertising</font> Local advertising budget for company at each location (in thousands of dollars)
* <font color="orange">Population</font> Population size in region (in thousands)
* <font color="orange">Price</font> Price company charges for car seats at each site
* <font color="orange">ShelveLoc</font> A factor with levels Bad, Good and Medium indicating the quality of the shelving location for the car seats at each site
* <font color="orange">Age</font> Average age of the local population
* <font color="orange">Education</font> Education level at each location
* <font color="orange">Urban</font> A factor with levels No and Yes to indicate whether the store is in an urban or rural location
* <font color="orange">US</font> A factor with levels No and Yes to indicate whether the store is in the US or not

Assume that we are going to build a linear regression model to predict <font color="orange">Sales</font> by using all the other variables as predictors. We are interested in analyzing which explanatory variables (i.e., predictors) are strongly associated with <font color="orange">Sales</font>.

### 2.1 Fit a multiple regression model to predict sales as a function of all the other variables, being careful to handle the qualitative variables.

Now it should be easy for you to build a multiple linear regression model to predict <font color="orange">sales</font> by using all the other variables as predictors. What you should pay attention to is how the <font color="blue"> lm() </font> treats categorical variables. What you should do is to fit the model and print the summary.

Print out the summary of your fitted linear model:

You have seen that the <font color='blue'>str()</font> shows that <font color='orange'>ShelveLoc</font> is a categorical variable (or factor), which can take values on "bad", "Medium" and "Good". To interpret a trinary variable, the linear model in R generates two binary variables, "ShelveLocGood" and "ShelveLocMedium". In other words, for a categorical variable that can take n values, we need to introduce n-1 binary variables.  

As discussed in Section 3.6.6 in the textbook, "the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/contrasts.html"><font color="blue">contrasts()</font></a> function returns the coding that R uses for the dummy variables."

<font color="blue">contrasts()</font></a> function sets and views the contrasts associated with a factor. 

In [ ]:
contrasts(Carseats$ShelveLoc) = contr.treatment(length(unique(Carseats$ShelveLoc)))
summary(lm(Sales~., data = Carseats))

Can you see the difference?

### 2.2 For which of the predictors can you reject the null hypothesis <font color="red">H0: j = 0</font>?

In order to answer this question, we need to look at the p-value in the summary, which is indicated by Pr(>|t|).
For the predictors which have the low p-value (less than 0.01), we can reject the null hypothesis. What are the predictors that have a strong association with the predictor?

Answer: 

### 2.3 On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome

In this task, we are going to build a new multiple linear regression model. The new model includes the predictors which had the significant association with the response variable in the previous model. We are going to explore the difference between a full model and a model with less but meaningful predictors. Here we will use the variables identified in Question 2.2.

Print out the summary of your fitted linear model:

### 2.4 How well do the models in <font color="red">2.1</font> and <font color="red">2.3</font> fit the data?

Now, we want to compare these two models. For this purpose, we examine the models using the following items:
* Look at the <font color="red">R-squared</font> and <font color="red">F</font> statistics and the corresponding p-values.
* Check the residuals with various residual plots.
* Perform F-tests by comparing the two models using the <font color="blue">anova()</font> function

As can be seen, there is not much difference between the full model and the one with less predictors. The multiple R-squared and the adjusted R-squared scores slightly drops. The trade-off between simple model and modeling accuracy is very small. In this case, we would like to choose the simple model.

Of course, you can also compare the diagnostic plots, for example

In [ ]:
par(mfcol=c(2,2))
plot(fit1, which = 1)
plot(fit1, which = 2)
plot(fit2, which = 1)
plot(fit2, which = 2)

The plots show that both models comply with the assumptions.
We can also use the <font color="blue">anova</font> function.

The p-value is quite large, so we cannot reject the null hypothesis. It is not statistically significant that the difference between the full model and the model with only the coefficients strongly associated with the response. In regard to the interpretability of the model, we'd prefer a model with less predictors.

### 2.5 Select the best regression variables with the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/step.html">step()</a> function.

Instead of manually selecting a subset of variables that are strongly associated with the predictor, R provides some automatic approaches of variable selection. Here, you are going to study how to use the <font color="blue">step()</font></a> function. This function can perform stepwise regression, either forward or backward. For more details, please refer to section 11.7 of "<a href="http://ebookcentral.proquest.com.ezproxy.lib.monash.edu.au/lib/monash/detail.action?docID=680900">R Cookbook"</a.>

The step() function should give you the same model as we just generated according to the P-Value.

As shown above, the best model is selected based on AIC after four steps. The best model has <font color="orange">CompPrice</font>, <font color="orange">Income</font>, <font color="orange">Advertising</font>, <font color="orange">Price</font>, <font color="orange">ShelveLoc</font> and <font color="orange">Age</font> as predictors. 

The best model learned by the <font color="blue">step()</font> function is exactly the same as model derived in Question 2.3. You can print out the summary of the model using the <font color="blue">summary()</font> function.

### 2.6 Expand the model in <font color="red">2.3</font> with interaction effects using the <font color="red">:</font> symbols. 
In order to further explore the data set and study the model, we want to investigate how the potential interactions between predictors can effect the linear model. Here we'd like to try <font color="blue">income: advertising</font>,  <font color="blue">price:age</font>, or both. Do any interactions appear to be statistically significant?

The function you used is the <font color="blue">update()</font> function. Remember to print out the summary of your fitted models.

In the ANOVA test, the small p-value indicates that the models are significantly different. If we set the significant code to 0.01, adding just Income: Advertising to the model derived in Question 2.3 makes a significant difference.

### 2.7 Using the model from <font color="red">2.6</font>, obtain 95% confidence intervals for the coefficients.

In [ ]:
confint(fit3, level = 0.95)

### 2.8 Is there evidence of outliers or high leverage observations in the model from <font color="red">2.6</font>?

There are many ways of checking if there are some outliers in your data. As discussed in the core textbook, we can use residual plots to identify outliers. Let's have a look

In [ ]:
plot(fit3, which = 3)

In the plot above, we can find some data points that stay a bit far from the majority of the points. Can we conclude that there are some outliers?

Let's use the <a href="http://artax.karlin.mff.cuni.cz/r-help/library/art/html/outlierTest.html"><font color="blue">outlierTest()</font></a> function to do an outlier test. The function will report the Bonferroni p-values for studentized residuals in linear regression models.

In [ ]:
library(car)

In [ ]:
outlierTest(fit3, cutoff=0.05, digits = 1)

The test shows that data point <font color='red'>358</font> is not an outlier.

Are there any influential data points? To answer this question, we can use the <font color="blue">influencePlot()</font> function. If influential points are identified, the function will also return a data frame with the hat values, Studentized residuals and Cook's distance.

In [ ]:
influencePlot(fit3, scale=5, id.method="noteworthy", main="Influence Plot", sub="Circle size is proportial to Cook's Distance" )

In the influence plot, the two dashed vertical lines are drawn at twice and three times the average hat value respectively. The three horizontal lines are drawn at -2, 0 and 2 on the Studentized-residual scale.